# Data analysis - Unalimented setup with varying temperature

This notebook uses code developed in the previous experiments to make data from a large collection of recordings.

Setup characteristics:

![Experimental setup](../docs/experimental-setup-spoon.png)

## Import libraries

In [1]:
import os
import re
import math

from scipy.signal import find_peaks

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

/home/theovld/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Open and parse data

## Merging all files

This is only necessary once, when the data is spread out in multiple files (so, at the output of the [MATLAB program](../1-video-treatment)

In [2]:
folder_path = 'results/26.11.not_filled'

In [ ]:
all_files = os.listdir(folder_path)

def create_df(pattern, name, data=None):
    """
        Utility function to create a dataframe out of sparse files
        :param pattern regexp corresponding to files to gather
        :param name the name of the merged file
        :param data if set, the parameter should be a dataframe containing at least columns name, temperature, beginning - This is usually the "information" dataframe containing all video meta
    """
    dfs = []

    files = [f for f in all_files if pattern.search(f)]

    for i, file in enumerate(files):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        filename = file[:12]
        df['filename'] = filename
        if data is not None:
            info = data[data['name'] == filename].iloc[0]
            df.drop(df[df.time < info.beginning].index, inplace=True)
            df['temperature'] = info.temperature
        dfs.append(df)

    df = pd.concat(dfs, ignore_index=True)
    df.to_csv(os.path.join(folder_path, f'{name}.csv'), index=False)

In [ ]:
info_pattern = re.compile(r'IMG_\d{4}.avi.csv')

create_df(info_pattern, 'info')

In [3]:
info_df = pd.read_csv(f'{folder_path}/info.csv')

In [ ]:
data_pattern = re.compile(r'IMG_\d{4}.avi.\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}.csv')

create_df(data_pattern, 'data', info_df)

In [4]:
data_df = pd.read_csv(f'{folder_path}/data.csv', usecols=['frame','time','area','tip_y','filename','temperature'])

## Data types

Temperature is a controlled parameter of our experiments

In [5]:
info_df['temperature'] = info_df['temperature'].astype(str).astype('category')
info_df.sort_values(by='temperature', ascending=True, inplace=True)

data_df['temperature'] = data_df['temperature'].astype(str).astype('category')
data_df.sort_values(['temperature', 'frame'], ascending=[True, True], inplace=True)

## Log-time, log-frame

In [6]:
data_df['log_frame'] = np.log(data_df['frame'])
data_df['log_time'] = np.log(data_df['time'])

## Converting area to cm²

139px = 6mm
So, 1px = 0,6/139 width and length, we square to get the area

In [7]:
px_to_cm = (0.6 / 139)

data_df['area_px'] = data_df['area'].copy()
data_df['area'] = data_df['area'] * px_to_cm**2
data_df['tip_y_px'] = data_df['tip_y'].copy()
data_df['tip_y'] = data_df['tip_y'] * px_to_cm

## Smoothing

As the smoothing is quite severe, some anomalous values can appear at the edges. For that, we fill the last rows with a constant value

In [8]:
from scipy.signal import savgol_filter

for column in ['area', 'area_px', 'tip_y', 'tip_y_px']:
    column_smooth = f'{column}_smooth'
    data_df[column_smooth] = savgol_filter(data_df[column], window_length=12, polyorder=3, mode='mirror')

    def fill_last_10_rows(g):
        # Check if the group has more than 10 rows
        if len(g) > 10:
            # Get the value before the last 10 rows
            last_valid_value = g.iloc[-11][column_smooth]
            # Assign this value to the last 10 rows
            g.iloc[-10:, g.columns.get_loc(column_smooth)] = last_valid_value
        return g
    
    data_df = data_df.groupby('temperature', group_keys=False).apply(fill_last_10_rows)

/tmp/ipykernel_29426/950131844.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data_df = data_df.groupby('temperature', group_keys=False).apply(fill_last_10_rows)
/tmp/ipykernel_29426/950131844.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_df = data_df.groupby('temperature', group_keys=False).apply(fill_last_10_rows)
/tmp/ipykernel_29426/950131844.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False t

# Definition of code utilities

## Unique color palette for graphs, difference by temperature

In [9]:
dark24_colors = plotly.colors.qualitative.Dark24
color_scale = px.colors.qualitative.Dark24

temperatures = data_df['temperature'].unique()
temp_to_color = {t: color_scale[i % len(color_scale)] for i, t in enumerate(temperatures)}

def get_color(index):
    return color_scale[index % len(color_scale)]

## Create projector-friendly graphs

In [10]:
def style_for_projector(fig, monochrome=False, font_size=1, line_width=3, marker_size=10, width=None, height=None, log_x=False, log_y=False):
    """Applies projector-friendly styling to an existing plotly figure"""

    fig.update_layout(
        template="plotly_white",
        title=dict(
            font=dict(size=24*font_size, color='black')
        ),
        width=width,
        height=height,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(t=100, b=100, l=100, r=50)
    )
    
    if line_width is not None:
        fig.update_traces(
            line_width=line_width,
            marker_size=marker_size,
        )

    if monochrome:
        fig.update_traces(line_color='black', marker_color='black')
    
    fig.update_xaxes(
        title=dict(font=dict(size=20*font_size)),
        tickfont=dict(size=16*font_size*0.75),
        showgrid=True,
        gridcolor='lightgray',
        nticks=6,
        color='black',
        minor=dict(ticks='inside', ticklen=6, showgrid=True)
    )
    
    if log_x:
        fig.update_xaxes(type='log')
    else:
        fig.update_xaxes(exponentformat='power', showexponent='last')
    
    fig.update_yaxes(title=dict(font=dict(size=20*font_size)),
        tickfont=dict(size=16*font_size*0.75),
        showgrid=True,
        gridcolor='lightgray',
        nticks=6,
        color='black',
        minor=dict(ticks='inside', ticklen=6, showgrid=True)
    )

    if log_y:
        fig.update_yaxes(type='log')
    else:
        fig.update_yaxes(exponentformat='power', showexponent='last')

    return fig

# Individual analysis

## Static area plot

In [ ]:
fig = px.line(data_df, x='time', y='area', color='temperature', log_x=True)
fig.update_layout(xaxis_title='Time (s)', yaxis_title='Area (cm^2)')

In [ ]:
video_name = 'IMG_0808.avi'

fig = px.line(data_df[(data_df['filename'] == video_name)], x='time', y='area')
fig = style_for_projector(fig, monochrome=True, font_size=2.5, width=1500)
fig.show()

Let's plot on a semi-log scale on time scale. We don't use log scale for area as we don't have a full decade 

In [ ]:
fig = px.line(data_df[(data_df['filename'] == video_name)], x='time', y='area')
fig = style_for_projector(fig, monochrome=True, font_size=2.5, width=1500, log_x=True)
fig.update_xaxes(range=[math.log10(3), 2])
fig.show()

In [ ]:
fig = px.line(data_df[(data_df['filename'] == video_name)], x='time', y='area_smooth')
fig = style_for_projector(fig, monochrome=True, font_size=2.5, width=1500, log_x=True)
fig.update_xaxes(range=[math.log10(13.4), 2])
fig.show()

## Tip position and breakup length

In [ ]:
data_df['tip_y_cm'] = data_df['tip_y'] * px_to_cm

In [ ]:
px.line(data_df[(data_df['filename'] == video_name)], x='time', y='tip_y_cm', labels={'x': 'Time (s)', 'y': 'Smoothed Area (cm²)'})

## Tool: create an animated side-by-side plot and video

In [ ]:
from matplotlib.animation import FuncAnimation, FFMpegWriter

start_time = 60.
end_time = 66.

# Set your desired frames per second
fps = info_df[info_df['filename'] == video_name].framerate.iloc[0]

df = data_df[(data_df['filename'] == video_name) & (data_df['time'] >= start_time) & (data_df['time'] <= end_time)]

# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 4))

line, = ax.plot([], [], lw=2)  # an empty line to update later

# Set axis labels and limits if desired
ax.set_xlim(df['time'].min(), df['time'].max())
ax.set_ylim(df['area'].min() - 0.1, df['area'].max() + 0.1)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Area (#{pixels})')
ax.set_title('Animated Line Plot')

# Initialization function: clear the line data
def init():
    line.set_data([], [])
    return line,

# Update function: draw the line, up to the ith frame
def update(frame):
    # frame will go from 0 to len(df)-1
    # Plot all data up to the current frame index
    x_data = df['time'].iloc[:frame+1]
    y_data = df['area'].iloc[:frame+1]
    line.set_data(x_data, y_data)

    return line,

# Create the animation
anim = FuncAnimation(fig, update, frames=len(df), init_func=init, blit=True, interval=1000/fps)

# Save the animation to a file
# Make sure you have ffmpeg installed. You can change codec and bitrate as needed.
writer = FFMpegWriter(fps=fps, codec='libx264', bitrate=1500)
anim.save('line_animation.mp4', writer=writer)

TODO:
- Define a crop height afterward

# Peaks analysis from Phase 3

## Qualitative observations from videos

- Plus la température est basse, plus les phases sont longues

- La phase 3 devient difficile à distinguer, mais on la définit qualitativement comme une **phase d'oscillation régulières** (il ne faut pas que ça soit une simple coupure dans un flot régulier)
- Sur les graphiques -> Point de départ = une grosse descente suivie de pics régulièrement espacés
- En général, avant la phase 3, la courbe a beaucoup de bruit et est + droite ; après, comporte surtout des oscillations
- Même avant que ça soit complètement coupé, les "drops" vont très souvent par pairs


## Parsing data for Phase 3

- We isolate data points that concern phase 3, that were manually identified on the videos
- We apply a strong smoothing on the area curve

In [12]:
phase3_map = dict(zip(info_df['temperature'], info_df['oscillations_beginning']))
phase3_df = data_df[data_df['time'] >= data_df['temperature'].map(phase3_map).astype('float')].copy()

# Threshold: only detect maximas that are above a certain value (not to catch too small maximas, that are not "drops" in terms of our definition)
mid_max = phase3_df.groupby('temperature')['area_px_smooth'].transform(lambda x: np.max(x) / 3)
threshold_df = phase3_df[phase3_df['area_px_smooth'] > mid_max]

/tmp/ipykernel_29426/1488171521.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mid_max = phase3_df.groupby('temperature')['area_px_smooth'].transform(lambda x: np.max(x) / 3)


## Finding absolute peaks

First, we apply the function from scipy to identify all peaks in the signals

In [13]:
peaks_i, _ = find_peaks(
    threshold_df['area_px_smooth'],
    prominence=2,
    distance=15,  # One peak for at least 1 second (empirical value)
)
peaks_df = threshold_df.iloc[peaks_i].copy()

peaks_df['temperature'] = peaks_df['temperature'].astype(str)
peaks_df['peak_number'] = peaks_df.groupby('temperature').cumcount()

## Filtering to get peaks corresponding to drops

Then, we filter peaks that are not "real", i.e. they do not stand enough to be considered a drop time (for instance, we don't want to catch the bounce after the drop)

In [14]:
time_window = 0.1
min_threshold = 1e4 * px_to_cm**2

def check_relative_height(temperature, time):
    area = data_df[(data_df['time'] >= time - time_window) & (data_df['time'] <= time + time_window) & (data_df['temperature'] == temperature)]['area_smooth']
    return area.max() - area.min() > min_threshold

relative_height_valid = peaks_df.apply(
    lambda row: check_relative_height(row['temperature'], row['time']), axis=1
)

valid_peaks_df = peaks_df[relative_height_valid].copy()
valid_peaks_df['peak_number'] = valid_peaks_df.groupby('temperature').cumcount()

Finally, we plot everything on a figure to control that our filtering is correct:

In [14]:
fig = px.line(
    phase3_df,
    x='time',
    y='area_smooth',
    color='temperature',
    color_discrete_sequence=color_scale
)

fig_scatter = px.scatter(
    valid_peaks_df,
    x='time',
    y='area_smooth',
    color='temperature',
    color_discrete_sequence=color_scale
)

for trace in fig_scatter.data:
    fig.add_trace(trace, row=1, col=1)

style_for_projector(fig)
fig.show()

NameError: name 'phase3_df' is not defined

# Standalone peaks analysis

Here we pick a particular temperature to study the characteristics of peaks from a same profile: time, shape, area...

## Selecting the video to analyse

In [ ]:
standalone_filename = 'IMG_0808.avi'

standalone_data_df = data_df[data_df['filename'] == standalone_filename].copy()
standalone_peaks_df = valid_peaks_df[valid_peaks_df['filename'] == standalone_filename].copy()

## Calculating data and plotting peaks together

In [ ]:
def calculate_slope(x, y):
  start_idx = np.where(y >= 0.2)[0][0]
  x_start, y_start = x[start_idx], y[start_idx]

  # Find the peak (maximum y value and its corresponding x value)
  peak_idx = np.argmax(y)  # Index of maximum y value
  x_peak, y_peak = x[peak_idx], y[peak_idx]
  return (y_peak - y_start) / (x_peak - x_start)

In [ ]:
right_range = 0.5
left_range = 0.5

normalization_y = 0.6

N = len(standalone_peaks_df)
n = np.arange(N)
integrals = np.zeros(N)
integrals_smooth = np.zeros(N)
slopes = np.zeros(N)
slopes_smooth = np.zeros(N)

fig = make_subplots(rows=2, cols=1)

for i, point in enumerate(standalone_peaks_df['time']):
  # Data fetching
  temp_df = standalone_data_df[(standalone_data_df['time'] >= point - right_range) & (standalone_data_df['time'] <= point + left_range)]
  x = temp_df['time'].values - point
  y = temp_df['area'].values
  y = (y - np.min(y)) / (np.max(y) - np.min(y))

  y_smooth = temp_df['area_smooth'].values
  y_smooth = (y_smooth - np.min(y_smooth)) / (np.max(y_smooth) - np.min(y_smooth))

  integrals[i] = np.trapz(y, x)
  integrals_smooth[i] = np.trapz(y_smooth, x)
  
  slopes[i] = calculate_slope(x, y)
  slopes_smooth[i] = calculate_slope(x, y_smooth)
  
  norm_point = x[y_smooth >= normalization_y][0]

  fig.add_trace(go.Scatter(
    x=x, y=y_smooth, mode='lines', name=f'Peak {i}', line=dict(
      color=get_color(i),
    ),
  ), row=1, col=1)

  fig.add_trace(go.Scatter(
    x=x / (-norm_point), y=y_smooth, mode='lines', name=f'Peak {i} with x norm', line=dict(
      color=get_color(i),
    ),
  ), row=2, col=1)

style_for_projector(fig, font_size=2.5, width=1200, height=1000)
fig.show()

## Integrals relation

In [ ]:
fig = make_subplots(rows=1, cols=1)

mask = (n != 10)
mean = np.mean(integrals[mask])
mean_smooth = np.mean(integrals_smooth[mask])

fig.add_trace(go.Scatter(
    x=n, y=integrals, mode='markers', name='Un-smoothed', marker_color='red'
))
fig.add_hline(y=mean, line_color='red', name='Un-smoothed')

fig.add_trace(go.Scatter(
    x=n, y=integrals_smooth, mode='markers', name='Smoothed', marker_color='blue'
))
fig.add_hline(y=mean_smooth, line_color='blue', name='Smoothed')

style_for_projector(fig, font_size=2.5)
fig.show()

## Slopes relation

In [ ]:
px.scatter(slopes_smooth)

In [ ]:
mask = (n != 10)

# fit = np.poly1d(np.polyfit(n[mask], slopes[mask], 3))

fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(
    x=n, y=slopes, name='Data', mode='markers', marker_color='black'
))

n_continuous = np.arange(0, N, 0.01)

# fig.add_trace(go.Scatter(
#     x=n_continuous, y=fit(n_continuous), name='Prediction', mode='lines', line_color='blue'
# ))

def f(x):
    v = 1.2 * (x-4.6)
    return 1 + 7.1*np.exp(v) / (1 + np.exp(v))

fig.add_trace(go.Scatter(
    x=n_continuous, y=f(n_continuous), name='Sigmoid', mode='lines', line_color='red'
))

style_for_projector(fig, monochrome=True, font_size=2.5)
fig.show()

# Joint peaks analysis

## Plotting stacked peak lines

In [28]:
delta = 0.5

temperatures = data_df['temperature'].unique()
for i, temperature in enumerate(temperatures):
    fig = make_subplots(rows=1, cols=1)

    for peak_num, peak_time in enumerate(valid_peaks_df.loc[valid_peaks_df['temperature'] == temperature, 'time']):
        seg = phase3_df[(phase3_df['temperature'] == temperature) &
                    (phase3_df['time'] >= peak_time - delta) &
                    (phase3_df['time'] <= peak_time + delta)].copy()
        
        x = seg['time'].values - peak_time
        y = seg['area_smooth'].values
        y_norm = (y - y.min()) / (y.max() - y.min())
        norm_point = x[y_norm >= 0.6][0]

        fig.add_trace(
            go.Scatter(
                x=x / (- norm_point),
                y=y_norm,
                mode='lines',
                name=f"Peak {peak_num} (Temp={temperature})",
                hovertemplate='Time: %{x}<br>Norm Area: %{y}<extra></extra>',
                marker=dict(color=get_color(peak_num))
            ),
        )

    fig = style_for_projector(fig, font_size=2.5)
    
    fig.update_layout(
        height=600,
        showlegend=False,
        title=f'T={temperature}°C',
        xaxis_title="Shifted t [s]",
        yaxis_title="Normalized A"
    )
    
    fig.write_image(f'{temperature}.png')

# Looking for the tricking time law

## Performing the regression

We use a linear regression on the logarithm of the time, in order to find an exponential law for the time.

In [21]:
temperatures = valid_peaks_df['temperature'].unique()

x_values = []
y_values = []
polys = []
residuals = []

for i, t in enumerate(temperatures):
    # Filter for a single temperature
    df_temp = valid_peaks_df[valid_peaks_df['temperature'] == t]

    # Extract x and y for the polyfit
    x = df_temp['peak_number'].values
    y = df_temp['log_time'].values
    
    x_values.append(x)
    y_values.append(y)

    # Compute linear fit: np.polyfit returns coefficients [m, b] for a line m*x+b
    z, residual, rank, singular_values, rcond = np.polyfit(x, y, 1, full=True)
    polys.append(z)
    residuals.append(residual)

## Results

### Joint area and predictions plot

We plot on the dashboard everything for our peaks analysis, and the verification of laws.

In [ ]:
color_scale = px.colors.qualitative.Dark24
temp_to_color = {t: color_scale[i % len(color_scale)] for i, t in enumerate(temperatures)}

# Initialize a Plotly figure
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"colspan": 2}, None],
           [{"colspan": 2}, None]],
    vertical_spacing=0.15,
    subplot_titles=("Area vs Log-time", "Log-Time vs. Peak Number by Temperature")
)

fig_scatter = px.scatter(
    peaks_df,
    x='log_time',
    y='area_smooth',
    color='temperature',
    color_discrete_sequence=color_scale
)

# Create a line plot for the studied phase
fig_lines = px.line(
    phase3_df,
    x='log_time',
    y='area_smooth',
    color='temperature',
    color_discrete_sequence=color_scale
)

for trace in fig_scatter.data:
    fig.add_trace(trace, row=1, col=1)

# Add all line traces to the specified subplot
for trace in fig_lines.data:
    fig.add_trace(trace, row=1, col=1)

for i, t in enumerate(temperatures):
    x = x_values[i]
    y = y_values[i]

    # Generate a smooth line for plotting
    x_line = np.linspace(min(x), max(x), 100)
    y_line = np.polyval(polys[i], x_line)

    # Add scatter points
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        name=f'Data (T={t}°C)',
        marker=dict(color=temp_to_color[t]),
        legendgroup=str(t),  # Grouping so toggling one also toggles the other if desired
        visible=True
    ),  row=2, col=1)

    # Add regression line
    fig.add_trace(go.Scatter(
        x=x_line,
        y=y_line,
        mode='lines',
        name=f'Fit (T={t}°C)',
        line=dict(color=temp_to_color[t]),
        legendgroup=str(t),
        visible=True
    ),  row=2, col=1)


style_for_projector(fig, line_width=None)
fig.update_layout(
    title='Log Time vs. Peak Number by Temperature',
    legend_title='Temperature Data',

    height=1000,

    xaxis_title='Log-Time',
    yaxis_title='Smoothed Area (cm²)',

    xaxis2_title='Peak Number',
    yaxis2_title='Log-Time',
)

fig.show()

### Temperature influence on parameters

In [26]:
fig = make_subplots(rows=1,cols=2)

slopes = [poly[0] for poly in polys]
intercepts = [poly[1] for poly in polys]

fig.add_trace(
    go.Scatter(x=temperatures, y=slopes, name='a', mode='markers', marker_color='black', showlegend=False),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=temperatures, y=np.exp(intercepts), name='t_0 = exp(b)', mode='markers', marker_color='black', showlegend=False),
    row=1, col=2
)
style_for_projector(fig, monochrome=True, font_size=2, width=1200)
fig.write_image('params.png')

### Plotting a standalone law

In [ ]:
video_name = 'IMG_0808.avi'
index = 13

x = x_values[index]
y = np.exp(y_values[index])
y_line = np.exp(np.polyval(polys[index], x))

fig = make_subplots(rows=1,cols=1)
fig.add_trace(
    go.Scatter(x=x, y=y, mode='markers'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=x, y=y_line, mode='lines'),
    row=1, col=1
)

fig.update_layout(xaxis_title='n', yaxis_title='t_n')
style_for_projector(fig, monochrome=True, font_size=2.5)